In [1]:
import pandas as pd
import praw
import os
from dotenv import load_dotenv
import boto3
import concurrent.futures
import time

In [8]:
load_dotenv()

client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')

In [24]:
reddit = praw.Reddit(
    client_id= client_id,
    client_secret= client_secret,
    user_agent= user_agent
)

In [27]:
# Function to fetch comments for a single submission
def fetch_comments(submission):
    submission.comments.replace_more(limit=0)
    return [
        {
            'comment': comment.body,
            'score': comment.score,
            'submission_title': submission.title,
        }
        for comment in submission.comments.list()
    ]

start_time = time.time()

# Define the subreddit and search query
subreddit_name = "soccer"
search_query = "Manchester United"
comments_data = []

# Scrape recent posts and their comments
submissions = reddit.subreddit(subreddit_name).search(search_query, sort='new', limit=50)  # Increase limit here

# Use ThreadPoolExecutor to fetch comments in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(fetch_comments, submissions)

# Collect all results into comments_data
for result in results:
    comments_data.extend(result)

# Limit to the first 100 comments
comments_data = comments_data[:100]

# Convert to DataFrame
comments_df = pd.DataFrame(comments_data)
csv_file_path = 'manchester_united_comments.csv'

# Save the comments DataFrame to CSV (if desired)
comments_df.to_csv(csv_file_path, index=False)

# Upload the CSV file to S3
s3_bucket_name = 'reddit-football-text'
s3_file_name = 'man_utd.csv'

s3 = boto3.client('s3')
s3.upload_file(csv_file_path, s3_bucket_name, s3_file_name)

# End timing
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Data uploaded to S3 successfully. Time taken: {elapsed_time:.2f} seconds.")

Data uploaded to S3 successfully. Time taken: 14.93 seconds.


In [2]:
from transformers import pipeline

# Load pre-trained sentiment-analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

ImportError: /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: symbol __nvJitLinkComplete_12_4, version libnvJitLink.so.12 not defined in file libnvJitLink.so.12 with link time reference

In [ ]:
!conda install -c conda-forge cudatoolkit=12.1
!conda install -c conda-forge cudnn

Retrieving notices: ...working... done